# Formulaires

## Introduction

`Laravel` ne possède pas nativement de mécanismede gestion de formulaires aussi sophistiqué que `Symfony`.
Les formulaires sont intégrés dans les squelettes `Blade`.

- La manière la plus courante de faire est d'écrire les formulaires en HTML.
- Il est possible d'utiliser la [bibliothèque `html`]() de LaravelCollective (mais plus mise à jour depuis 2019)
- Il existe aussi une [bibliothèque](https://github.com/Momenoor/form-builder) qui porte les formulaires `Symfony` dans `Laravel`


## Création de formulaire

Par défaut, Laravel n'offre pas procédure particulière pour créer des formulaires. Par conséquent, on a tendance à créer de simples formulaires HTML dous forme de fichiers `Blade`.

Ces formulaires contiennent des variables, en particulier pour définirla route de retour.

> **Note** Une bonne pratique consistera à créer deux routes séparées pour les versants `GET` et `POST` du formulaire, par exemple :
>
> - Route::get('product/add', ...);
> - Route::post('product/new', ...);

### Sécurisation

La chose la plus importante est l'intégration du jeton de sécurité qui vise à emp^cher les **attaques CSRF**. Pour cela, il faut ajouter la directive :

%%blade

<form method="POST" action="{{ route(product.new) }}">
@csrf
</form>

###  Méthodes HTTP

Il est possible d'utiliser toutes les méthodes HTTP disponibles,et il est bien sûr recommandé de le faire. Pour cela, nous devrons intégrer deux éléments dans notre code :

1. La déclaration de la méthode dans le formulaire,ce qui se fait avec une directive ad hoc `@method` :

In [ ]:
%%blade

<form action="{{ route('users.update',['id' => $id]) }}">
    @csrf
    @method('PATCH')
</form>

2. Naturellement, déclarer une route qui accepte la méthode :

In [ ]:
%%php

Route::patch('users/{id}', 'UsersController@update')->name('users.update');
// ou
Route::resource('users', 'UsersController');

## Les formulaires « à la Symfony »

`Symfony` gère les formulaires de manière très différente.

1. D'une part, les caractéristiques des formulaires sont décrites dans une classe
2. La vue ne fait que déclarer les noms des champs, laissant l'environnement engendrer le code HTML

Si vous voulez adopter cette manière de faire, vous devrez installer la bibiothèque requise, comme :

In [ ]:
%%bash

composer require momenoor/form-builder

Vous bénéficiez alors d'un nouvelle commande `make` :

%%bash

./artisan make:form Forms/ProductForm

Vous avez maintenant une nouvelle classe qui est une représentation abstraite du formulaire. Celui-ci est construit par une méthode `buildForm`, dont l'action principale est de lister les champs à afficher via la méthode `add`.

In [ ]:
%%php

namespace App\Forms;

use Momenoor\FormBuilder\FormValidator;

class ProductForm extends FormValidator
{
    public function buildForm()
    {
        $this->add('name', 'text');
        
        // Autres champs ..

        $this->addDefaultActions();
    }
}

`add` définit un champ au trvers de trois arguments :
- un nom, qui doit correspondre à une propriété de l'entité
- un type, qui doit aussi correspondre au type de la propriété et qui servira à la création du wifget HTML
- un ensemble d'options.

Par exemple :

In [ ]:
%%php

public function buildForm()
{
    $this->add('status', 'choice', [
        'choices'     => StaticLabel::status(),
        'empty_value' => _('-'),
        'validation'  => 'required',
        'label'       => _('Status')
    ])
}


Nous déclarons ici un champ de type `Choice`, à savoir une liste déroulante, sont les valeurs sont issues d'une classe particulière, du modèle par exemple, avec un label, une valeu par défaut et une méthode de validation.

> **Note** Comme vous pouvez le remarquer, les classes de formulaires héritent de la classe `FormValidator` — ce qui est en soi un peu curieux — et la validation se définit directement dans la classe. Elle se fera du côé du navigateur, ce qui n'exclut pas la validation coté serveur que nous verrons après.

La documentation de la bibliothèque fournit d'autres exemples de définitions de champs.

Il est possible en particulier de définir des sous-formulaires :

In [ ]:
%%php

$this->add('user', 'form', [
   'label' => _('User'),
   'icon'  => 'user',
   'class' => \FormBuilder::create('Momenoor\Expendable\Forms\User\UserForm', [
       'model'                  => $this->getUserModel(),
       'do_not_display_role_id' => true
   ])
]);

Une fois le formulaire créé, nous puvons l'utiliser dansun contrôleur :

In [ ]:
%%php

class SongsController extends BaseController {

    public function create(FormBuilder $formBuilder)
    {
        $form = $formBuilder->create(\App\Forms\SongForm::class, [
            'method' => 'POST',
            'url' => route('song.store')
        ]);

        return view('song.create', compact('form'));
    }

    public function store(FormBuilder $formBuilder)
    {
        $form = $formBuilder->create(\App\Forms\SongForm::class);

        if (!$form->isValid()) {
            return redirect()->back()->withErrors($form->getErrors())->withInput();
        }

        // Suite du code...
    }
}

Et il faut encore l'intégrer dans le squelette `Blade` correspondant :

%%blade

@extends('app')

@section('content')
    {!! form($form) !!}
@endsection

## LaravelCollective

Il existe enfin une troisième méthode de construction de formulaire avec la bilbliothèque `html` de LaravelCollective.

Celle-ci doit néanmoins être utilisée avec précaution, car elle ne semble plus activement maintenue depuis quelques années.

Elles s'appuie sur une syntaxe mixte PHP/HTML, plus proche du style natif de `Laravel`. Ci-dessous, nous affichons par par exemple un champ de type `password` ayant une classe CSS `secret`:

In [ ]:
%%blade

{!! Form::password('password', ['class' => 'secret']); !!}

> **Note** Comme on le remarque, nous demandons à Blade de ne pas protéger le code HTML produit.

Pour créer un formulaire complet, il faudra intégrer dans un squelette `Blade` :

In [ ]:
%%blade

{!! Form::open(['route' => ['route.name', $user->id]], 'method' => 'PUT') !!}
{!! Form::token() !!}
{!! Form::label('email', 'Adresse électronique') !!}
{!! Form::text('email', '', ['class' => 'text-field']) !!}
{!! Form::label(['city', 'Ville') !!}
{!! Form::select('city', '['75000' => 'Paris', '59000' => 'Lille', '33000' => 'Bordeaux']', ['class' => 'select-field']) !!}
{!! Form::close() !!}
        

## Ressources

- [Documentation de Laravel]()
- [Documentation de Momenoor](https://github.com/Momenoor/form-builder)
- [Documentation de Kistijan Husak](https://github.com/kristijanhusak/laravel-form-builder)
- [Documentation de Laravelcollective](https://laravelcollective.com/docs/6.x/html)